In [17]:
# === ETL Pipeline: Sales Performance Analysis ===

import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

# === Step 1: Extract ===
# Simulating reading the CSV data
data = {
    'country': ['USA', 'USA', 'USA', 'USA', 'USA',
                'India', 'India', 'India', 'India', 'India',
                'Germany', 'Germany', 'Germany', 'Germany', 'Germany'],
    'year': [2020, 2021, 2022, 2023, 2024,
             2020, 2021, 2022, 2023, 2024,
             2020, 2021, 2022, 2023, 2024],
    'total_sales': [500000, 550000, 600000, 650000, 700000,
                    200000, 220000, np.nan, 270000, 290000,
                    300000, 320000, 350000, 370000, np.nan]
}

raw_df = pd.DataFrame(data)

print("=== Raw Sales Data ===")
print(raw_df)

# === Step 2: Preprocessing ===

# Handling missing sales data
# Impute missing 'total_sales' by interpolation (better for time series)
raw_df['total_sales'] = raw_df.groupby('country')['total_sales'].transform(lambda x: x.interpolate())

print("\n=== After Imputation (Interpolating Missing Sales) ===")
print(raw_df)

# === Step 3: Transformation ===

# Calculate Year-over-Year (YoY) Growth % for each country
raw_df['previous_sales'] = raw_df.groupby('country')['total_sales'].shift(1)
raw_df['yoy_growth_pct'] = ((raw_df['total_sales'] - raw_df['previous_sales']) / raw_df['previous_sales']) * 100

# Scale 'total_sales' and 'yoy_growth_pct' between 0 and 1 for comparison
scaler = MinMaxScaler()

# We only scale meaningful rows (skip rows where previous_sales is NaN)
scaled_features = scaler.fit_transform(raw_df[['total_sales', 'yoy_growth_pct']].fillna(0))

raw_df[['total_sales_scaled', 'yoy_growth_pct_scaled']] = scaled_features

print("\n=== After Transformation (YoY Growth and Scaling) ===")
print(raw_df)

# Drop helper column
processed_df = raw_df.drop(columns=['previous_sales'])

# === Step 4: Load ===

processed_df.to_csv('processed_sales_data.csv', index=False)

print("\nProcessed sales data saved to 'processed_sales_data.csv'")

=== Raw Sales Data ===
    country  year  total_sales
0       USA  2020     500000.0
1       USA  2021     550000.0
2       USA  2022     600000.0
3       USA  2023     650000.0
4       USA  2024     700000.0
5     India  2020     200000.0
6     India  2021     220000.0
7     India  2022          NaN
8     India  2023     270000.0
9     India  2024     290000.0
10  Germany  2020     300000.0
11  Germany  2021     320000.0
12  Germany  2022     350000.0
13  Germany  2023     370000.0
14  Germany  2024          NaN

=== After Imputation (Interpolating Missing Sales) ===
    country  year  total_sales
0       USA  2020     500000.0
1       USA  2021     550000.0
2       USA  2022     600000.0
3       USA  2023     650000.0
4       USA  2024     700000.0
5     India  2020     200000.0
6     India  2021     220000.0
7     India  2022     245000.0
8     India  2023     270000.0
9     India  2024     290000.0
10  Germany  2020     300000.0
11  Germany  2021     320000.0
12  Germany  2022     